# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json`

In [83]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
! mongoimport --type json -d uk_food -c establishments --drop --jsonArray Resources/establishments.json

2024-05-24T08:50:05.630-0700	connected to: mongodb://localhost/
2024-05-24T08:50:05.631-0700	dropping: uk_food.establishments
2024-05-24T08:50:06.430-0700	39779 document(s) imported successfully. 0 document(s) failed to import.


In [84]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [86]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'autosaurus', 'classDB', 'config', 'epa', 'fruits_db', 'local', 'met', 'test', 'uk_food']


In [87]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [88]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [89]:
# review the collections in our new database
# This instruction is the same as the preceding cell. 
print(db.list_collection_names())

['establishments']


In [90]:
# review a document in the establishments collection
pprint(db.establishments.find())

In [91]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [92]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [93]:
# Insert the new restaurant into the collection
establishments.insert_one(new_restaurant)

In [94]:
# Check that the new restaurant was inserted
query = {"BusinessName":"Penang Flavours"}
result = establishments.find_one(query)
pprint(result)

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6650b78b09ffe3900b1e9cb8'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [95]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query= {}
fields = {"BusinessType": "Restaurant/Cafe/Canteen", 
          "BusinessTypeID":1}
results = establishments.find_one(query, fields)
print(results)

{'_id': ObjectId('6650b72d8d922f2f620c70dc'), 'BusinessTypeID': 1, 'BusinessType': 'Restaurant/Cafe/Canteen'}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [96]:
# Update the new restaurant with the correct BusinessTypeID
db.establishments.update_one({"BusinessName":"Penang Flavours"},
                                 {"$set":
                                      {'BusinessTypeID': "7843"}
                                 }
                            )

In [97]:
# Confirm that the new restaurant was updated
query= {}
fields = {"BusinessName":"Penang Flavours", 
          "BusinessTypeID":1}
results = establishments.find_one(query, fields)
print(results)

{'_id': ObjectId('6650b72d8d922f2f620c70dc'), 'BusinessTypeID': 1, 'BusinessName': 'Penang Flavours'}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [98]:
# Check if any remaining documents include Dover
query = {"LocalAuthorityName" : "Dover"}
print("The number of documents that have LocalAuthorityName of Dover: ", establishments.count_documents(query)) 

The number of documents that have LocalAuthorityName of Dover:  994


In [99]:
# Delete all documents where LocalAuthorityName is "Dover"
query = {"LocalAuthorityName":"Dover"}
establishments.delete_many(query)


In [100]:
# Check if any remaining documents include Dover
query = {"LocalAuthorityName":"Dover"}
print("The number of documents that have LocalAuthorityName of Dover: ", establishments.count_documents(query)) 

The number of documents that have LocalAuthorityName of Dover:  0


In [101]:
# Check that other documents remain with 'find_one'
query= {}
fields = {"LocalAuthorityName":1}
results = establishments.find_one(query, fields)
print(results)

{'_id': ObjectId('6650b72d8d922f2f620c73c3'), 'LocalAuthorityName': 'Folkestone and Hythe'}


5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [ ]:
# NOTE TO GRADER: Prior to the updating of data types that follows, an initial review using MongoDB Compass reveals the presence
# of four documents with (1) geocode.latitiue and geocode.longitude string data types, and (2) "RatingValue" of "AwaitingPublication."
# One of thesee that is printed here for confirmation of the changes.  
# As follows, quotation marks around "AwaitingPublication" is indicative of a string data type.

query = {"BusinessName":"Ashford Pizza and Kebab House"}
result = establishments.find_one(query)
pprint(result)

In [ ]:
# Change the data type from String to Decimal for longitude and latitude
establishments.update_many({}, [{'$set':{'geocode.latitude':{'$toDecimal':'$geocode.latitude'}}}])
establishments.update_many({}, [{'$set':{'geocode.longitude':{'$toDecimal':'$geocode.longitude'}}}])

Use `update_many` to convert `RatingValue` to integer numbers.

In [ ]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection",
               "Awaiting Inspection", 
               "AwaitingPublication", 
               "Pass", 
               "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [ ]:
# Change the data type from String to Integer for RatingValue
establishments.update_many({}, [ {'$set':
                                  {"RatingValue" : {'$toInt' : "$RatingValue"}}} 
                               ]
                          )

In [ ]:
# Check that the coordinates and rating value are now numbers
# As commented above, checking the change to (1) 'geocode.latitude' and 'geocode.longitude' and (2) 'RatingValue'
# First, confirming change to "None"


pprint("Confirming the change to <None>:")
query = {"BusinessName":"Ashford Pizza and Kebab House"}
result = establishments.find_one(query)
pprint(result)

The preceding check confirms the changes to the data types of (1) String to Decimal for "geocode.longitude" and "geocode.latitude", and (2) String to Integer for "RatingValue" reflected in the change from 'AwaitingPublication' to "None"; the absence of quotation marks around "None" as printed above is indicative of its change to an integer data type.